# Homework 2

Austin Gill

Problems:
* 3: 2
* 4: 3, 13, 14
* Text Addition

## Problem 3.2

First, the usual two-link manipulator kinematics code.

In [ ]:
# %load prob3.6/kinematics.py
"""
Runs forward and inverse kinematics for the two link manipulator
"""
import numpy as np


def forward(a1, a2, q1, q2):
    """Forward kinematics for a configuration point"""
    x = a2 * np.cos(q1 + q2) + a1 * np.cos(q1)
    y = a2 * np.sin(q1 + q2) + a1 * np.sin(q1)

    return x, y


def inverse(a1, a2, x, y):
    """Inverse kinematics for a workspace point"""
    D = (x**2 + y**2 - a1**2 - a2**2) / (2 * a1 * a2)
    q2 = np.arctan2(np.sqrt(1 - D**2), D)
    q1 = np.arctan2(y, x) - np.arctan2(a2 * np.sin(q2), a1 + a2 * np.cos(q2))

    return q1, q2

Then write a Python class for each of the three desired ROS2 nodes.

In [ ]:
# %load prob3.6/nodes.py
import itertools

import matplotlib.pyplot as plt
import numpy as np
import rclpy as ros
import seaborn as sns
from std_msgs.msg import Float64MultiArray

from kinematics import forward, inverse


class WorkspaceGenerator:
    """A ROS2 computation node to generate workspace points and publish them
    to the `physData` topic as a `Float64MultiArray` message.

    The messages will be published at 5Hz.
    """

    def __init__(self, name, loop=False):
        """Create a ROS2 computation node to generator workspace points.

        :param name: The name of the computation node.
        :param loop: Whether the generator should loop to the beginning
                     if/when it finishes generating points.
        """
        self.node = ros.create_node(name)
        self.publisher = self.node.create_publisher(Float64MultiArray, 'physData')
        # Running every 0.2 seconds is 5Hz.
        self.timer = self.node.create_timer(0.2, self.callback)
        self.msg = Float64MultiArray()
        self.loop = loop
        # A generator of points to publish
        self.workspace_points = self.points()

    def points(self):
        """Returns a generator of workspace points."""
        x = np.linspace(0, 10, 100)
        y = 15 - x

        workspace_points = ((xi, yi) for xi, yi in zip(x, y))

        if self.loop:
            return itertools.cycle(workspace_points)

        return workspace_points

    def callback(self):
        """Publishes a new workspace point every time this function is called."""
        try:
            self.msg.data = next(self.workspace_points)
            self.publisher.publish(self.msg)
        except StopIteration:
            pass


class ConfigGenerator:
    """A ROS2 computation node to generate configuration space points and
    publish them to the `thetaData` topic as a `Float64MultiArray` message.

    This node subscribes to workspace points from the `physData` topic.
    """

    def __init__(self, name):
        """Create a ROS2 computation node to generator config space points.

        :param name: The name of the computation node.
        """
        self.node = ros.create_node(name)
        self.publisher = self.node.create_publisher(Float64MultiArray,
                                                    'thetaData')
        self.subscriber = self.node.create_subscription(Float64MultiArray,
                                                        'physData',
                                                        self.callback)

    def callback(self, msg):
        """Publishes a new workspace point every time this function is
        called.

        :param msg: The message this node should handle.
        """
        a1, a2 = 10, 10
        x, y = msg.data
        t1, t2 = inverse(a1, a2, x, y)
        msg.data = [t1, t2]
        self.publisher.publish(msg)


class KinematicsVerifier:
    """A ROS2 computation node to generate configuration space points and
    publish them to the `thetaData` topic as a `Float64MultiArray` message.

    This node subscribes to workspace points from the `physData` topic.
    """

    def __init__(self, name):
        """Create a ROS2 computation node to verify our inverse kinematics.

        :param name: The name of the computation node.
        """
        self.node = ros.create_node(name)
        self.workspace_subscriber = self.node.create_subscription(Float64MultiArray,
                                                                  'physData',
                                                                  self.workspace_cb)
        self.config_subscriber = self.node.create_subscription(Float64MultiArray,
                                                               'thetaData',
                                                               self.config_cb)
        # Set a nicer color scheme for matplotlib.
        sns.set()
        plt.figure()
        self.wplot, = plt.plot([], '.', color='green', label='workspace points')
        self.cplot, = plt.plot([], '.', color='blue', label='computed workspace points')
        plt.title('Verified workspace points')
        plt.legend()
        plt.ion()
        plt.show()
        # BUG: Matplotlib window becomes unresponsive (so I can't click "save") when points are
        #      finished plotting. This is because the event loop is not "spun" for lack of a better
        #      word.
        # FIX: When script is done executing, call plt.show(block=True) to continue running the MPL
        #      event loop so that the "save" and "close" buttons, etc., work.

    def update_wplot(self, xp, yp):
        """Updates the physical workspace plot.

        :param xp: A single x point to add to the plot.
        :param yp: A single y point to add to the plot.
        """
        x, y = self.wplot.get_data()
        x = np.append(x, xp)
        y = np.append(y, yp)
        self.wplot.set_data(x, y)
        self.wplot.axes.set_xlim(np.amin(x)-0.1, np.amax(x)+0.1)
        self.wplot.axes.set_ylim(np.amin(y)-0.1, np.amax(y)+0.1)
        plt.pause(0.001)

    def update_cplot(self, xp, yp):
        """Updates the verified workspace plot

        :param xp: A single x point to add to the plot.
        :param yp: A single y point to add to the plot.
        """
        x, y = self.cplot.get_data()
        x = np.append(x, xp)
        y = np.append(y, yp)
        self.cplot.set_data(x, y)
        self.cplot.axes.set_xlim(np.amin(x)-0.1, np.amax(x)+0.1)
        self.cplot.axes.set_ylim(np.amin(y)-0.1, np.amax(y)+0.1)
        plt.pause(0.001)

    def workspace_cb(self, msg):
        """Gets called for each workspace point.

        :param msg: The msg containing a workspace point.
        """
        x, y = msg.data
        self.update_wplot(x, y)

    def config_cb(self, msg):
        """Gets called for each configuration point

        :param msg: The msg containing a configuration point.
        """
        a1, a2 = 10, 10
        t1, t2 = msg.data
        x, y = forward(a1, a2, t1, t2)
        self.update_cplot(x, y)

Then you can use an `rclpy.executors.SingleThreadedExecutor` to run the three nodes at once from the same thread.

In [ ]:
# %load prob3.6/main.py
#!/usr/bin/env python3
import matplotlib.pyplot as plt
import rclpy as ros
from rclpy.executors import SingleThreadedExecutor

from nodes import ConfigGenerator, KinematicsVerifier, WorkspaceGenerator


def main():
    """A single main entry point for a collection of ROS nodes."""
    ros.init(args=None)

    kv = KinematicsVerifier(name='kinematics_verifier')
    cg = ConfigGenerator(name='config_generator')
    wg = WorkspaceGenerator(name='workspace_generator')

    # Runs all three nodes in a single thread so that there's a single entry point and I don't have
    # to topologically sort and run each python script individually.
    executor = SingleThreadedExecutor()

    executor.add_node(kv.node)
    executor.add_node(cg.node)
    executor.add_node(wg.node)

    try:
        executor.spin()
    except KeyboardInterrupt:
        # Did someone say "hacky-bandaide-bugfix"??
        print('Waiting for matplotlib figure to close...')
        plt.show(block=True)

    # TODO: There's probably some OOPy way to handle this automagically, but I'm too pissed off at
    #       matplotlib to care right now.
    kv.node.destroy_node()
    cg.node.destroy_node()
    wg.node.destroy_node()
    ros.shutdown()

My original solution had a lot of hacky code that the `SingleThreadedExecutor` wiped out.

```python
try:
    # Run each ROS node in its own process
    kv = Process(target=KinematicsVerifier.create_and_run, args=('kinematics_verifier',))
    cg = Process(target=ConfigGenerator.create_and_run, args=('config_generator',))
    wg = Process(target=WorkspaceGenerator.create_and_run, args=('workspace_generator',))

    # Make sure consumers are created before the producers
    kv.start()
    cg.start()
    wg.start()

except KeyboardInterrupt:
    kv.join()
    cg.join()
    wg.join()
```

Where `create_and_run` was a method each node class defined that created an instance of the class, and immediately called `rclpy.spin()` to run the node. Note though that I could not get the Matplotlib plot working when I tried to use an `rclpy.executors.MultiThreadedExecutor`, though there was data being published to the `/physData` and `/thetaData` topics.

This produces the following plot. Note how the green points are not visible--due to the blue points being plotted on top.

![plot](prob3.6/prob3.6.svg)

## Problem 4.3

Using ROS and Python, write a program to calculate the motion of a differential drive robot.

1. Write a program that publishes a sequence of wheel velocities on the topic `/WheelVel` at 10Hz. Use the multiarray datatype. This node should be named `Control`. This program should also publish on a topic named `/Active` either `1` or `0` at 1 Hz to say whether or not the robot is active (meaning done with wheel velocities and you can plot now: active =1, done = 0). Demonstrate the code on

   $$\dot\phi_1=2+2e^{-t_n}$$
   $$\dot\phi_2=2+e^{-2t_n}$$

   for $0 \leq t \leq 10$.

2. Write a program that uses the differential drive kinematics to derive the robot linear and angular velocities. Publish the velocities using the ROS standard twist message and name the topic `/RobotVel`. This node should be named `ForwardK`. Assume that $D=10$, $L=20$, and the robot starts at $(0,0,0)$.

3. Write a program that will subscribe to the twist message and plot the robot’s path using Python plotting when it gets the signal on the `Active` topic. This node should be named `RobotPlot`.

First, we have the usual kinematics code:

In [ ]:
# %load prob4.3/kinematics.py
import numpy as np


def x_dot(r, w1, w2, theta):
    """x time derivative for diff. drive forward kinematics"""
    return (r / 2) * (w1 + w2) * np.cos(theta)

def y_dot(r, w1, w2, theta):
    """y time derivative for diff. drive forward kinematics"""
    return (r / 2) * (w1 + w2) * np.sin(theta)

def theta_dot(r, L, w1, w2):
    """theta time derivative for diff. drive forward kinematics"""
    return (r / (2 * L)) * (w1 - w2)

Then we create the three ROS nodes

In [ ]:
# %load prob4.3/nodes.py
from collections import deque

import matplotlib.pyplot as plt
import numpy as np
import rclpy as ros
import seaborn as sns
from geometry_msgs.msg import Twist
from std_msgs.msg import Bool, Float64MultiArray

from kinematics import theta_dot, x_dot, y_dot


class Control:
    """A ROS2 computation node that publishes wheel velocities for a differential drive robot.

    The node publishes the velocities to `/WheelVel` at 10Hz, and publishes a 0 to `Active` to
    indicate the node is done publishing to `/WheelVel`.
    """

    def __init__(self, name, step):
        """Creates the ROS2 computation node to publish wheel velocities.

        :param name: The name of the node.
        :param step: The problem timestep.
        """
        self.node = ros.create_node(name)
        self.publisher = self.node.create_publisher(Float64MultiArray, 'WheelVel')
        self.active_pub = self.node.create_publisher(Bool, 'Active')
        # Running every 0.1 seconds is 10Hz.
        self.timer = self.node.create_timer(0.1, self.velocity_cb)
        self.active_timer = self.node.create_timer(1, self.active_cb)
        self.wheel_velocities = self.velocities(step)

    @staticmethod
    def velocities(step):
        """Returns the wheel velocities as parameterized in the problem."""
        t = np.arange(0, 10+step, step)
        phi1 = 2 + 2 * np.exp(-t)
        phi2 = 2 + np.exp(-2 * t)

        return deque(zip(phi1, phi2))

    def velocity_cb(self):
        """Publishes wheel velocities at 10Hz"""
        try:
            msg = Float64MultiArray()
            msg.data = self.wheel_velocities.popleft()
            self.publisher.publish(msg)
        except IndexError:
            pass

    def active_cb(self):
        """Indicates whether the Control node is still active"""
        msg = Bool()
        msg.data = bool(self.wheel_velocities)
        self.active_pub.publish(msg)


class ForwardK:
    """A ROS2 computation node that runs forward kinematics on the wheel velocities of a
    differential drive robot. This node publishes the forward kinematics on the topic /RobotVel as
    it receives the wheel velocities.
    """

    def __init__(self, name, step):
        """Creates the ROS2 computation node to run the forward kinematics.

        :param name: The name of the node.
        :param step: The problem timestep.
        """
        self.node = ros.create_node(name)
        self.subscriber = self.node.create_subscription(Float64MultiArray, 'WheelVel', self.callback)
        self.publisher = self.node.create_publisher(Twist, 'RobotVel')
        self.step = step

    def callback(self, msg):
        """Receives wheel velocities and runs the forward kinematics."""
        phi1, phi2 = msg.data
        dtheta = theta_dot(r=5, L=20, w1=phi1, w2=phi2)
        theta = dtheta * self.step
        dx = x_dot(r=5, w1=phi1, w2=phi2, theta=theta)
        dy = y_dot(r=5, w1=phi1, w2=phi2, theta=theta)

        msg = Twist()
        msg.linear.x = dx
        msg.linear.y = dy
        msg.angular.x = dtheta

        self.publisher.publish(msg)


class RobotPlot:
    """A ROS2 node to numerically integrate and plot the path of a differential drive robot."""

    def __init__(self, name, step):
        """Creates the ROS2 node.

        :param name: The name of the ROS2 node.
        :param step: The problem timestep.
        """
        self.node = ros.create_node(name)
        self.twist_sub = self.node.create_subscription(Twist, 'RobotVel', self.twist_cb)
        self.active_sub = self.node.create_subscription(Bool, 'Active', self.active_cb)
        # A flag to indicate whether we're finished or not.
        self.finished = False
        self.step = step
        # A list of (dx, dy, dtheta) velocities as np.arrays.
        self.velocities = []
        # A list of (x, y, theta) positions as np.arrays.
        self.positions = []

    def twist_cb(self, msg):
        """Receives the twist messages."""
        # Don't worry about spreading the numerical integration over multiple calls to the callback.
        self.velocities.append(np.array([msg.linear.x, msg.linear.y, msg.angular.x]))

    def integrate(self):
        """Performs simple numerical integration to convert a list of velocities, a known time step,
        and an initial position to a list of position points to plot.
        """
        # Start at (0, 0, 0)
        x0 = np.array([0, 0, 0])
        self.positions.append(x0)
        for v in self.velocities:
            # Update the position by multiplying the derivatives elementwise with the time step
            x0 = x0 + v * self.step
            self.positions.append(x0)

    def plot(self):
        """Plots the position points"""
        x = [p[0] for p in self.positions]
        y = [p[1] for p in self.positions]
        sns.set()
        plt.plot(x, y)
        plt.title('Differential Drive Computed Path')
        plt.xlabel('$x$')
        plt.ylabel('$y$')
        plt.show()

    def active_cb(self, msg):
        """Receives the active messages."""
        # If not active, then done generating points.
        if not msg.data and not self.finished:
            self.integrate()
            self.plot()

            # TODO: Find a way to cause the entire program (the executor) to terminate?
            # self.node.executor.stuff()

            self.finished = True

Then we can run the three nodes

In [ ]:
# %load prob4.3/main.py
#!/usr/bin/env python3
import rclpy as ros
from rclpy.executors import SingleThreadedExecutor
from nodes import Control, ForwardK, RobotPlot

STEP_SIZE = 0.1

def main():
    """A single main entry point for a collection of ROS nodes."""
    ros.init(args=None)

    control = Control(name='Control', step=STEP_SIZE)
    forward_k = ForwardK(name='ForwardK', step=STEP_SIZE)
    robot_plot = RobotPlot(name='RobotPlot', step=STEP_SIZE)

    # Runs all three nodes in a single thread so that there's a single entry point and I don't have
    # to topologically sort and run each python script individually.
    executor = SingleThreadedExecutor()

    executor.add_node(robot_plot.node)
    executor.add_node(forward_k.node)
    executor.add_node(control.node)

    try:
        executor.spin()
    except KeyboardInterrupt:
        pass

    control.node.destroy_node()
    forward_k.node.destroy_node()
    robot_plot.node.destroy_node()
    ros.shutdown()

Which produces the following plot of the robot's motion.

![prob4.3 motion](prob4.3/prob4.3.svg)

Which corresponds with my mental expectation of what the robot should do. (On my first attempt, I had `self.velocities.append(np.array([msg.linear.x, msg.linear.x, msg.angular.x]))` instead of `self.velocities.append(np.array([msg.linear.x, msg.linear.y, msg.angular.x]))`, which did not produce a plot anything like what I expected)

## Problem 4.13

## Problem 4.14